<div class="alert alert-info">

**Note:**    WARNING: This notebook uses a new, early release Chameleon python API. The notebook and API will likely be updated without notice. To request a new python feature, please create a help desk ticket. 
    

</div>

## Tutorial: Stitching Networks between Chameleon Sites

This notebook demonstrates how to use ExoGENI to stitch networks between Chameleon sites. It will walk through reserving and configuring stitchable networks on each Chameleon site and using the xoStitch tool to create a layer2 circuit between the networks. 

#### Modules 

- [Reserve Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)
- [Get Network by Name](../modules-python/network/get_network_by_name.ipynb)

TODO: add the rest of the module links when they are ready

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [ ]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project
chi.set('project_name', 'ExoGENI@Chameleon') # Replace with your project name

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair

#GENI Pem file
geni_pem_file='/home/pruth/work/geni-pruth.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")+'8'
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_skylake"
server_count=2

#Network Config
physical_network='exogeni'
cidr='192.168.42.0/24'

## Create Chicago Network and Server

#### Create a Lease at Chicago

In [ ]:
#Set the region
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
#add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
#add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
uc_lease = get_lease_by_name(lease_name)
    
#Print the lease info
print(json.dumps(uc_lease, indent=2))

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
#Get the lease by name
uc_lease = get_lease_by_name(lease_name)

uc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', uc_lease['reservations']))[0]['id']
uc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', uc_lease['reservations']))[0]['id']
uc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', uc_lease['reservations']))[0]['id']

print("uc_compute_reservation_id: " + uc_compute_reservation_id)
print("uc_network_reservation_id: " + uc_network_reservation_id)
print("uc_floatingip_reservation_id: " + uc_floatingip_reservation_id)

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
chi.set('region_name', 'CHI@UC')  
#Get the network
network = get_network_by_name(network_name)
print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
uc_network_vlan = network['provider:segmentation_id']
print('uc_network_vlan: ' + str(uc_network_vlan))


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [ ]:
subnet = add_subnet(subnet_name, network_name, cidr=cidr)

print(json.dumps(subnet, indent=2))

#### Add a Router

TODO: add links here

In [ ]:
router = create_router(router_name, gw_network_name='public')

print(json.dumps(router, indent=2))

#### Attach the Router and Subnet

TODO: Add links here

In [ ]:
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

## Create TACC Network and Server

#### Create a Lease at TACC

In [ ]:
#Set the region
chi.set('region_name', 'CHI@TACC')     # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
tacc_lease = get_lease_by_name(lease_name)
    
#Print the lease info
print(json.dumps(tacc_lease, indent=2))

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
#Get the lease by name
tacc_lease = get_lease_by_name(lease_name)

tacc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', tacc_lease['reservations']))[0]['id']
tacc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', tacc_lease['reservations']))[0]['id']
tacc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', tacc_lease['reservations']))[0]['id']

print("tacc_compute_reservation_id: " + tacc_compute_reservation_id)
print("tacc_network_reservation_id: " + tacc_network_reservation_id)
print("tacc_floatingip_reservation_id: " + tacc_floatingip_reservation_id)

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
chi.set('region_name', 'CHI@TACC')  
#Get the network
network = get_network_by_name(network_name)
print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
tacc_network_vlan = network['provider:segmentation_id']
print('tacc_network_vlan: ' + str(tacc_network_vlan))


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [ ]:
subnet = add_subnet(subnet_name, network_name, cidr=cidr)

print(json.dumps(subnet, indent=2))

#### Add a Router

TODO: add links here

In [ ]:
router = create_router(router_name, gw_network_name='public')

print(json.dumps(router, indent=2))

#### Attach the Router and Subnet

TODO: Add links here

In [ ]:
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

## Start the Servers

#### Start the UC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
chi.set('region_name', 'CHI@UC')  
#create the server
server = create_server(server_name, 
                       reservation_id=uc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
chi.set('region_name', 'CHI@UC')  
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

#### Start the TACC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
chi.set('region_name', 'CHI@TACC')

print("server_name " + server_name)
print("tacc_compute_reservation_id " + tacc_compute_reservation_id)
print("key_name " + key_name) 
print("network_name " + network_name)
print("image_name " + image_name)
print("flavor_name " + flavor_name)

#create the server
server = create_server(server_name, 
                       reservation_id=tacc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
chi.set('region_name', 'CHI@TACC')  

floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

#### Check the Status of the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

### Delete TACC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.set('region_name', 'CHI@TACC')

In [ ]:
delete_server_by_name(server_name)

#### De-configure Network
TODO: break up into steps

In [ ]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

In [ ]:
delete_router_by_name(router_name)

In [ ]:
delete_subnet_by_name(subnet_name)

In [ ]:
delete_network_by_name(network_name)

#### Release Lease

In [ ]:
delete_lease_by_name(lease_name)

### Delete UC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.set('region_name', 'CHI@UC')

In [ ]:
delete_server_by_name(server_name)

#### De-configure Network
TODO: break up into steps

In [ ]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

In [ ]:
delete_router_by_name(router_name)

In [ ]:
delete_subnet_by_name(subnet_name)

In [ ]:
delete_network_by_name(network_name)

#### Release Lease

In [ ]:
delete_lease_by_name(lease_name)